<img src='letscodebr_cover.jpeg' align='left' width=100%/>

# Aulas 4: GeoPandas - GeoDataFrame.

In [29]:
# !pip install geodatasets
# !pip install geopandas

In [30]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import descartes

###  Intro

O [Geopandas](https://geopandas.org/getting_started/introduction.html) é uma biblioteca que permite trabalhar com dados geoespaciais em Python. Para fazer isso, ele usa recursos de outras bibliotecas Python.

- Estende a funcionalidade do [pandas](https://towardsdatascience.com/geospatial-adventures-step-2-pandas-vs-geopandas-16e842d0e3a7). Em outras palavras, todas as operações que se aplicam a *Series* e *DataFrame* podem ser realizadas.

- Fornece uma interface com a biblioteca [matplotlib](https://towardsdatascience.com/mapping-with-matplotlib-pandas-geopandas-and-basemap-in-python-d11b57ab5dac) para gerar mapas. As figuras geométricas podem ser facilmente mapeadas com o método `plot()`.

- Permite operações em dados geométricos que representam dados espaciais (pontos, linhas e polígonos). Para isso, faz uso da biblioteca [shapely](https://www.learndatasci.com/tutorials/geospatial-data-python-geopandas-shapely/).

- Facilita o acesso aos dados em múltiplos *formatos de arquivo*, ampliando as operações fornecidas pela biblioteca [fiona](https://pythonrepo.com/repo/Toblerity-Fiona-python-geolocation)

- Faz projeções, ou seja, permite utilizar diferentes tipos de coordenadas para referenciar a posição geográfica dos dados geoespaciais.

Vamos analisar cada um desses pontos com mais detalhes.

### GeoSeries e GeoDataFrame

Assim como *Pandas* tem *Series* e *Dataframe* como estrutura de dados, o *Geopandas* tem algo semelhante chamado [Geoseries](https://geopandas.org/docs/reference/geoseries.html) e [GeoDataFrame](https://geopandas.org/gallery/create_geopandas_from_pandas.html).

A diferença entre eles é que **GeoSeries** e **GeoDataFrame** devem conter pelo menos uma coluna com tipos de dados geoespaciais. Esta coluna é chamada por padrão de [geometry](https://automating-gis-processes.github.io/site/notebooks/L2/geopandas-basics.html).

A coluna *geometry* contém uma forma geométrica que representa a posição espacial do objeto. Por exemplo, a posição espacial de uma empresa - suas coordenadas - são expressas pela forma geométrica *ponto*.

As formas geométricas são:

- POINT. Um ponto
- LINESTRING. Uma linha
- POLYGON. Uma superfície

Coleções de formas também podem ser representadas:

- MULTIPOINT. Vários pontos
- MULTILINESTRING. Múltiplas linhas
- MULTIPOLYGON. Múltiplas superfícies

Em [geodatos](https://www.geodatos.net/en/coordinates) você pode pesquisar as coordenadas das cidades do mundo.

### Cidades

Começamos gerando um `dataframe Pandas` com os dados da cidade. Observe que os dados geográficos de latitude e longitude são incluídos como *números*, para localizar cidades no espaço.

In [31]:
df_cidade = pd.DataFrame(
    {'Cidade': ['São Paulo', 'Rio de Janeiro', 'Salvador', 'Belo Horizonte', 'Manaus','Campo Grande', 'Florianópolis'],
     'Estado': ['SP', 'RJ', 'BA', 'MG', 'AM', 'MS', 'SC'],
     'Latitude':  [-23.5558, -22.9068, -12.9777, -19.9191, -3.1190, -20.4649, -27.5948],
     'Longitude': [-46.6396, -43.1729, -38.5016, -43.9387, -60.0217, -54.6218, -48.5569]})

In [32]:
type(df_cidade)

pandas.core.frame.DataFrame

In [33]:
df_cidade.dtypes

Cidade        object
Estado        object
Latitude     float64
Longitude    float64
dtype: object

Como a latitude e a longitude são numéricas, elas devem ser transformadas em uma forma geométrica para poder representá-las como dados geoespaciais.

O Geopandas faz isso com o método [`geopandas.points_from_xy()`](https://geopandas.org/docs/reference/api/geopandas.points_from_xy.html), gerando a forma geométrica **POINT**.

**IMPORTANTE:** este método possui dois parâmetros, o primeiro, $x$, deve receber a **longitude**, e o segundo, $y$, a deve receber **latitude**. Você pode logicamente pensar que deveria ser o contrário.

In [34]:
geometria = gpd.points_from_xy(df_cidade.Longitude, 
                               df_cidade.Latitude
                              )
geometria

<GeometryArray>
[ <POINT (-46.64 -23.556)>, <POINT (-43.173 -22.907)>,
 <POINT (-38.502 -12.978)>, <POINT (-43.939 -19.919)>,
  <POINT (-60.022 -3.119)>, <POINT (-54.622 -20.465)>,
 <POINT (-48.557 -27.595)>]
Length: 7, dtype: geometry

Agora transformamos o `dataframe` `df_cidade` em um `geoDataFrame` chamado `geo_bra`; os dados do dataframe são combinados com a variável recém-criada *geometry* (as coordenadas da cidade como uma forma geométrica).

Lembre-se de que os tipos de dados `GeoDataFrame` requerem pelo menos uma coluna com uma forma geométrica. Recomenda-se chamá-lo de *geometry*, pois é o padrão para salvar este tipo de dados.

In [35]:
geo_bra = gpd.GeoDataFrame(df_cidade, 
                           geometry = geometria
                          )

In [36]:
geo_bra.dtypes

Cidade         object
Estado         object
Latitude      float64
Longitude     float64
geometry     geometry
dtype: object

In [37]:
geo_bra

,Cidade,Estado,Latitude,Longitude,geometry
0,São Paulo,SP,-23.5558,-46.6396,POINT (-46.6396 -23.5558)
1,Rio de Janeiro,RJ,-22.9068,-43.1729,POINT (-43.1729 -22.9068)
2,Salvador,BA,-12.9777,-38.5016,POINT (-38.5016 -12.9777)
3,Belo Horizonte,MG,-19.9191,-43.9387,POINT (-43.9387 -19.9191)
4,Manaus,AM,-3.1190,-60.0217,POINT (-60.0217 -3.119)
5,Campo Grande,MS,-20.4649,-54.6218,POINT (-54.6218 -20.4649)
6,Florianópolis,SC,-27.5948,-48.5569,POINT (-48.5569 -27.5948)


### Países

Cada pais se representa por sus límites, o contornos. Dentro del tipo de datos geoespacial, usa la forma *POLYGON* o *MULTIPOLYGON*. Geopandas permite leerlos desde un archivo especial llamado *naturalearth_lowres*

Este archivo se guarda en una variable del tipo GeoDataFrame, ya que contiene una columna geometry con los polígonos de cada páis, sus contornos.

Cada país é representado por suas fronteiras ou contornos. No tipo de dados geoespaciais, use o formato *POLYGON* ou *MULTIPOLYGON*. O `Geopandas` permite a leitura de um arquivo especial chamado `naturalearth_lowres`.

Este arquivo é salvo em uma variável do tipo `GeoDataFrame`, pois contém uma coluna geométrica com os polígonos de cada país, seus contornos.

Por que alguns países precisam de um MULTIPOLIGONO em vez de simplesmente um POLIGONO?

### Referências


- [GeoPandas 0.9.0](https://geopandas.org/index.html)

- [The GeoPandas Cookbook](https://www.martinalarcon.org/2018-12-31-d-geopandas)

- [The Shapely User Manual](https://shapely.readthedocs.io/en/stable/manual.html)